# Concatenador Secondary Vegetation

Se importan las librerías necesarias para manejar datos:
* pandas: buscar archivos en carpetas
* glob: trabajar con rutas y archivos del sistema
* os:  procesar cadenas con expresiones regulares
* re: Esto permite cargar, filtrar y combinar múltiples archivos CSV de manera automatizada.

In [ ]:
import pandas as pd
import glob
import os
import re

Combina automáticamente todos los archivos CSV de “Secondary Vegetation” correspondientes a los años 1985 a 2024 en un único archivo consolidado. Primero busca todos los archivos que coinciden con el patrón definido y extrae el año de cada archivo, agregándolo como una columna adicional. Luego carga cada CSV en un DataFrame, acumula todos los DataFrames en una lista y los concatena en un solo DataFrame combinado, ordenado cronológicamente por año. Finalmente, guarda este dataset consolidado en secondary_vegetation_completo_1985_2024.csv.

In [2]:
def unir_secondary_vegetation_anual():
    archivos = glob.glob("Secondary Vegetation • Annual by class • *.csv")
    archivos.sort()
    
    datos_combinados = []
    
    for archivo in archivos:
        año = extraer_año_secondary_vegetation(archivo)
        if año is None:
            continue
        df = pd.read_csv(archivo)
        df['year'] = año
        datos_combinados.append(df)
    
    if not datos_combinados:
        return None
    
    df_combinado = pd.concat(datos_combinados, ignore_index=True)
    df_combinado = df_combinado.sort_values('year').reset_index(drop=True)
    return df_combinado

def extraer_año_secondary_vegetation(nombre_archivo):
    nombre_base = os.path.basename(nombre_archivo)
    patrones = [
        r'•\s*(\d{4})\.csv$',
        r'\s(\d{4})\.csv$',
        r'•.*•\s*(\d{4})\.csv$',
        r'Vegetation.*•.*•\s*(\d{4})\.csv$',
    ]
    for patron in patrones:
        match = re.search(patron, nombre_base)
        if match:
            año = int(match.group(1))
            if 1980 <= año <= 2030:
                return año
    return None

df_secondary_vegetation_completo = unir_secondary_vegetation_anual()

if df_secondary_vegetation_completo is not None:
    df_secondary_vegetation_completo.to_csv("secondary_vegetation_completo_1985_2024.csv", index=False)
